In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

### 데이터 병합

In [2]:
# 채널정보 파일 읽기
channel_df = pd.read_parquet('open/concat/2018_승인매출정보.parquet')

# 회원정보 파일 읽기
member_df  = pd.read_parquet('open/concat/2018_회원정보.parquet')

# 회원정보에서 ID와 Segment 컬럼만 추출
member_seg = member_df[['ID', 'Segment']]

# 중복된 ID 개수 확인
dup_count = member_seg['ID'].duplicated().sum()
print(f'중복된 ID 개수: {dup_count}')

# 중복된 ID를 첫 번째 항목만 남기고 제거
member_seg_unique = member_seg.drop_duplicates(subset='ID', keep='first')
print(f'중복 제거 후 행 수: {len(member_seg_unique)}')

# 채널정보에 Segment 컬럼 병합 (1:1 조인 보장)
df = channel_df.merge(member_seg_unique, on='ID', how='left')

# 결과 확인
print("병합 후 데이터프레임 크기:", df.shape)
print(df.head())

# 병합된 파일 저장
df.to_parquet('2018_승인매출정보_with_segment.parquet', index=False)

중복된 ID 개수: 2500000
중복 제거 후 행 수: 500000
병합 후 데이터프레임 크기: (3000000, 407)
     기준년월            ID  최종이용일자_기본  최종이용일자_신판  최종이용일자_CA  최종이용일자_카드론  \
0  201807  TRAIN_000000   20180719   20180713   20180719       10101   
1  201807  TRAIN_000001   20180719   20180719   20170728    20170327   
2  201807  TRAIN_000002   20180706   20180706   20180706    20151119   
3  201807  TRAIN_000003   20180721   20180715   20180721       10101   
4  201807  TRAIN_000004   20180124   20180124      10101       10101   

   최종이용일자_체크  최종이용일자_일시불  최종이용일자_할부  이용건수_신용_B0M  ...  승인거절건수_BL_B0M  \
0   20180203    20180709   20180713           11  ...              0   
1      10101    20180719   20171231           13  ...              0   
2   20141230    20180706   20180627           12  ...              0   
3   20141111    20180704   20180715            6  ...              0   
4   20180512    20180124      10101           -2  ...              0   

   승인거절건수_입력오류_B0M  승인거절건수_기타_B0M  승인거절건수_R3M  승인거절건수_한도초과_R3M  

In [3]:
# 데이터를 불러온다.
df = pd.read_parquet('2018_승인매출정보_with_segment.parquet')

### 값이 하나 뿐인 컬럼과 값이 모두 0인 컬럼도 제외

In [4]:
# 전부 0인 컬럼 찾기
zero_cols = [c for c in df.columns if df[c].eq(0).all()]

print("모두 0인 컬럼 (제거 대상):")
print(zero_cols)

# 전부 0인 컬럼 제거
df.drop(columns=zero_cols, inplace = True)

# 4) 결과 확인
print("최종 DataFrame shape:", df.shape)

모두 0인 컬럼 (제거 대상):
['이용건수_부분무이자_B0M', '이용금액_부분무이자_B0M', '여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M', '할부금액_부분_3M_R12M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '이용개월수_당사페이_R6M', '이용금액_당사페이_R6M', '이용금액_당사기타_R6M', '이용건수_당사페이_R6M', '이용건수_당사기타_R6M', '이용금액_당사페이_R3M', '이용금액_당사기타_R3M', '이용건수_당사페이_R3M', '이용건수_당사기타_R3M', '이용금액_당사페이_B0M', '이용금액_당사기타_B0M', '이용건수_당사페이_B0M', '이용건수_당사기타_B0M', '승인거절건수_입력오류_B0M', '승인거절건수_기타_B0M']
최종 DataFrame shape: (3000000, 378)


In [5]:
# 추출: 모든 값이 동일한(상수) 컬럼 찾기
const_cols = [c for c in df.columns if df[c].nunique(dropna=False) == 1]

# 계산: 제거 대상 컬럼 목록 출력
print("모두 같은 값인 컬럼 (제거 대상):")
print(const_cols)

# 생성: 상수 컬럼 제거
df.drop(columns=const_cols, inplace=True)

# 확인: 최종 DataFrame 상태 출력
print("최종 DataFrame shape:", df.shape)

모두 같은 값인 컬럼 (제거 대상):
['RP후경과월_유선방송', 'RP후경과월_건강']
최종 DataFrame shape: (3000000, 376)


In [6]:
# 전체 컬럼 리스트 생성
cols = df.columns.tolist()

# 최빈값 비율이 99.9% 초과하는 컬럼 수집
low_variance_cols = []
for col in cols:
    # 컬럼별 최빈값 비율 확인
    top_ratio = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_ratio > 0.999:
        low_variance_cols.append(col)

# 제거 대상 컬럼 목록 출력
print("제거 대상 컬럼:", low_variance_cols)

# 제거 대상 컬럼 삭제
df.drop(columns=low_variance_cols, inplace=True)

# 최종 컬럼 리스트 출력
print("최종 컬럼 리스트:", df.columns.tolist())

제거 대상 컬럼: ['이용건수_카드론_B0M', '이용금액_카드론_B0M', '이용건수_부분무이자_R12M', '이용건수_부분무이자_R3M', '이용개월수_부분무이자_R3M', '교통_통행료이용금액', '할부건수_14M_R12M', '할부건수_유이자_14M_R12M', '할부건수_무이자_14M_R12M', '할부건수_부분_12M_R12M', '할부금액_부분_6M_R12M', '할부금액_부분_14M_R12M', 'RP건수_학습비_B0M', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_학습비_전월', '신청건수_ATM_CL_B0', '신청건수_ATM_CL_R6M', '승인거절건수_BL_B0M', '승인거절건수_입력오류_R3M']
최종 컬럼 리스트: ['기준년월', 'ID', '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA', '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '이용건수_신용_B0M', '이용건수_신판_B0M', '이용건수_일시불_B0M', '이용건수_할부_B0M', '이용건수_할부_유이자_B0M', '이용건수_할부_무이자_B0M', '이용건수_CA_B0M', '이용건수_체크_B0M', '이용금액_일시불_B0M', '이용금액_할부_B0M', '이용금액_할부_유이자_B0M', '이용금액_할부_무이자_B0M', '이용금액_CA_B0M', '이용금액_체크_B0M', '이용후경과월_신용', '이용후경과월_신판', '이용후경과월_일시불', '이용후경과월_할부', '이용후경과월_할부_유이자', '이용후경과월_할부_무이자', '이용후경과월_부분무이자', '이용후경과월_CA', '이용후경과월_체크', '이용후경과월_카드론', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_할부_유이자_R12M', '이용건수_할부_무이자_R12M', '이용건수_CA_R12M', '이용건수_체크

### 결측치 처리

In [7]:
# 전체 행 개수 계산
total_rows = len(df)

# 각 컬럼의 결측치 개수 계산
missing_count = df.isna().sum()

# 결측치 비율 계산 (전체 행 대비 %)
missing_pct = (missing_count / total_rows * 100).round(2)

# 결측치 정보 데이터프레임으로 정리
missing_info = pd.DataFrame({
    'missing_count': missing_count,
    'missing_pct': missing_pct
})

# 결측치가 있는 컬럼만 출력
print("\n결측치 정보 (개수 및 전체 대비 비율 %):")
print(missing_info[missing_info['missing_count'] > 0])


결측치 정보 (개수 및 전체 대비 비율 %):
                missing_count  missing_pct
_1순위업종                 676567        22.55
_2순위업종                1143812        38.13
_3순위업종                1387704        46.26
_1순위쇼핑업종              1155434        38.51
_2순위쇼핑업종              1421400        47.38
_3순위쇼핑업종              1643222        54.77
_1순위교통업종              1457454        48.58
_2순위교통업종              2071153        69.04
_3순위교통업종              2556184        85.21
_1순위여유업종              2485573        82.85
_2순위여유업종              2878107        95.94
_3순위여유업종              2972290        99.08
_1순위납부업종              1522257        50.74
_2순위납부업종              2543911        84.80
_3순위납부업종              2888898        96.30
최종카드론_금융상환방식코드        2447897        81.60
최종카드론_신청경로코드          2448009        81.60
최종카드론_대출일자            2485868        82.86
Segment                600000        20.00


### 날짜형, 순서형, 수치형 컬럼 찾아서 분리 후 처리하기

In [8]:
# 검사할 컬럼 리스트
c1 = list (df.columns)
cols_to_check = c1

# 비수치형 컬럼 저장용 리스트
non_numeric_cols = []

# 각 컬럼에 대해 수치형 변환 시도 후 NaN 비율로 판단
for col in cols_to_check:
    # 결측을 빈 문자열로, 나머지를 문자열로 변환
    s = df[col].fillna('').astype(str)
    # 숫자로 변환 (불가능한 항목은 NaN)
    num = pd.to_numeric(s, errors='coerce')
    # 변환 불가 비율 계산
    na_ratio = num.isna().mean()
    # 일정 비율 이상이면 비수치형으로 간주 (여기선 100% 미수치형도 포함)
    if na_ratio > 0:
        non_numeric_cols.append(col)

# 결과 출력
print("비수치형(또는 혼합형) 컬럼:", non_numeric_cols)

비수치형(또는 혼합형) 컬럼: ['ID', '_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', '최종카드론_대출일자', '이용금액대', 'Segment']


In [9]:
# 각 컬럼별 분포(value_counts) 출력
c1 = list (df.columns)
for col in c1:
    print(f"▶ [{col}] 분포")
    print(df[col].value_counts(dropna=False))
    print()  # 빈 줄로 구분

▶ [기준년월] 분포
기준년월
201807    500000
201808    500000
201809    500000
201810    500000
201811    500000
201812    500000
Name: count, dtype: int64

▶ [ID] 분포
ID
TRAIN_000000    6
TRAIN_333343    6
TRAIN_333341    6
TRAIN_333340    6
TRAIN_333339    6
               ..
TRAIN_166662    6
TRAIN_166661    6
TRAIN_166660    6
TRAIN_166659    6
TEST_99999      6
Name: count, Length: 500000, dtype: int64

▶ [최종이용일자_기본] 분포
최종이용일자_기본
20180731    149387
20180930    127899
20180831    125873
20181130    117204
10101       116608
             ...  
20160120         1
20160126         1
20141216         1
20151208         1
20151121         1
Name: count, Length: 1292, dtype: int64

▶ [최종이용일자_신판] 분포
최종이용일자_신판
20180731    149641
20180930    128068
20180831    126047
10101       117610
20181130    117247
             ...  
20160120         1
20151208         1
20151101         1
20151206         1
20151029         1
Name: count, Length: 1277, dtype: int64

▶ [최종이용일자_CA] 분포
최종이용일자_CA
10101       2043433

In [10]:
# 이름에 '일자'가 들어가는 모든 컬럼 찾기
date_cols = [c for c in df.columns if '일자' in c]

# 각 일자 컬럼에 대해 _dt 컬럼 생성
for col in date_cols:
    # 문자열로 변환 후, '.0'이 있으면 제거
    s = df[col].astype(str).str.replace(r'\.0$', '', regex=True)
    # YYYYMMDD 포맷으로 파싱 (파싱 실패 시 NaT)
    df[f'{col}_dt'] = pd.to_datetime(s, format='%Y%m%d', errors='coerce')

# 원본 일자 컬럼 삭제
df.drop(columns=date_cols, inplace=True)

In [11]:
# 이용금액대 순서형 인코딩 → '_ord' 생성 & 원본 삭제
ord_levels_amt = [
    "09.미사용",
    "05.10만원-",
    "02.50만원+",
    "04.10만원+",
    "03.30만원+",
    "01.100만원+"
]
mapping_amt = {lvl: i+1 for i, lvl in enumerate(ord_levels_amt)}
df['이용금액대_ord'] = df['이용금액대'].map(mapping_amt)
df.drop(columns=['이용금액대'], inplace=True)

In [12]:
# 저장 직전의 컬럼과 0/NaN 컬럼 재확인
zero_cols = [c for c in df.columns if df[c].eq(0).all()]
na_cols   = [c for c in df.columns if df[c].isna().any()]

print("저장 직전 0 전부 컬럼:", zero_cols)
print("저장 직전 NaN 있는 컬럼:", na_cols)
print("저장 직전 전체 컬럼 수:", len(df.columns))

저장 직전 0 전부 컬럼: []
저장 직전 NaN 있는 컬럼: ['_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', 'Segment', '최종이용일자_기본_dt', '최종이용일자_신판_dt', '최종이용일자_CA_dt', '최종이용일자_카드론_dt', '최종이용일자_체크_dt', '최종이용일자_일시불_dt', '최종이용일자_할부_dt', '최종카드론_대출일자_dt']
저장 직전 전체 컬럼 수: 356


In [13]:
# Parquet 파일로 저장
df.to_parquet('승인매출정보_전처리.parquet', engine='pyarrow', index=False)

In [14]:
# 저장된 파일 불러와서 확인하기
df2 = pd.read_parquet('승인매출정보_전처리.parquet')
zero_after = [c for c in df2.columns if df2[c].eq(0).all()]
na_after   = [c for c in df2.columns if df2[c].isna().any()]

print("로드 후 0 전부 컬럼:", zero_after)
print("로드 후 NaN 있는 컬럼:", na_after)
print("로드 후 전체 컬럼 수:", len(df2.columns))

로드 후 0 전부 컬럼: []
로드 후 NaN 있는 컬럼: ['_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', 'Segment', '최종이용일자_기본_dt', '최종이용일자_신판_dt', '최종이용일자_CA_dt', '최종이용일자_카드론_dt', '최종이용일자_체크_dt', '최종이용일자_일시불_dt', '최종이용일자_할부_dt', '최종카드론_대출일자_dt']
로드 후 전체 컬럼 수: 356
